In [1]:
import requests
from dotenv import load_dotenv
import os
import pandas as pd
import time


In [2]:
load_dotenv()

True

In [3]:
METASLEUTH_API_KEY = os.getenv('METASLEUTH_API_KEY')
test_address = 'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU'
test_wallet_timeseries = pd.read_csv('../data/test_acc_timeseries.csv')

test_wallet_timeseries['timestamp'] = pd.to_datetime(test_wallet_timeseries['timestamp'])

In [4]:
url = 'https://aml.blocksec.com/address-label/api/v3/chain-list'
headers = {'API-KEY':METASLEUTH_API_KEY}

In [5]:
# import requests

# response = requests.get(
#     "https://aml.blocksec.com/address-label/api/v3/chain-list",
#     headers={"API-KEY":METASLEUTH_API_KEY},
# )

# data = response.json()

In [6]:
# print(data['data'])

In [7]:
def get_solana_chainid(data):
    for d in data['data']:
        chain_id = d.get('chain_id')
        chain_name = d.get('chain_name')

        if chain_name.lower() == 'solana':
            return {chain_id:chain_name}

In [8]:
# solana_metadata = get_solana_chainid(data)


In [9]:
# list(solana_metadata.keys())[0]

In [10]:
unique_receivers = test_wallet_timeseries['receiver'].dropna().unique()
unique_senders = test_wallet_timeseries['sender'].dropna().unique()
unique_senders


array(['5Hr7wZg7oBpVhH5nngRqzr5W7ZFUfCsfEhbziZJak7fr',
       'FLiPgGTXtBtEJoytikaywvWgbz5a56DdHKZU72HSYMFF',
       'FLiPGqowc82LLR173hKiFYBq2fCxLZEST5iHbHwj8xKb',
       '6UgXZZBoydXRNX6SLjwFDUUgn19GhjBpvNAXU3iepzUV',
       'GUq7PhyAUZko2mPhv3CupmdJKQ61LH8VyrdsRL25q7zg',
       'AGPZnBZUxmhAtcp8XjT4n8bCia9dEYhhm16M2sfFvmTU',
       '7DyZQw3iV5zhHssnNA6Nopi5zc8NGLbYjHMcaok6NN66',
       'DkNrNXmjDSmTB8vNnZh2FMf2sd29wsmfKDAUb2GHPEKS',
       'fLiPgg2yTvmgfhiPkKriAHkDmmXGP6CdeFX9UF5o7Zc',
       'Habp5bncMSsBC3vkChyebepym5dcTNRYeg2LVG464E96',
       '7Q7iiGK4ycigQ6AVhVN2tAA8djsh8iQ6pMx7eZ7xEygM',
       'GUWM1arUyDnkMGCHvJu3yt1qomJ988utqC3dFN2AUCDT',
       '4xDsmeTWPNjgSVSS1VTfzFq3iHZhp77ffPkAmkZkdu71',
       '5PXvR1byXDQtKhm8yYmRE3gw1yS6C5SYeGMU3AW6QK8c',
       '44F8FNq6xVWTBKKLbr3BJgAdkTFSDJSC5NbAShW1RyVZ',
       '6qCVVuwhShy4C5u6SVHREYA5uUww1RhBeQWBpwGqmovB',
       '9gY37M5s8jjA4shxZCBuWzqiAX49bdSiWbKmZ6ka3ec',
       '2EEw1A49utRqUsnYRYWtnpz2UQ7n7sJn8EEpdN2nKqWQ',
       '4bFV

In [11]:
unique_addresses = list(set(unique_receivers.tolist() + unique_senders.tolist()))
len(unique_addresses)

103

In [12]:
def chunked(iterable, size):
    """Yield successive `size`-sized chunks from iterable."""
    for i in range(0, len(iterable), size):
        yield iterable[i:i + size]

def fetch_address_labels(unique_addresses, chain_id, api_key, delay=0.2):
    url = "https://aml.blocksec.com/address-label/api/v3/batch-labels"
    headers = {
        "API-KEY": api_key,
        "Content-Type": "application/json"
    }
    results = []

    for chunk in chunked(unique_addresses, 100):
        response = requests.post(
            url,
            headers=headers,
            json={
                "chain_id": chain_id,
                "addresses": chunk
            }
        )
        data = response.json()
        if data.get("code") == 200000:
            results.extend(data.get("data", []))
        else:
            print(f"[Error] Batch failed: {data}")
        time.sleep(delay)  # optional delay to avoid rate limits

    return results

In [13]:
solana_labels = fetch_address_labels(
    unique_addresses=unique_addresses,
    chain_id=-3,  # SOLANA
    api_key=METASLEUTH_API_KEY
)


[Error] Batch failed: {'request_id': '70b0a882b04848a3', 'code': 400012, 'message': 'Daily request limit exceeded. Please try again tomorrow.'}
[Error] Batch failed: {'request_id': '1e497b64c99645f7', 'code': 400012, 'message': 'Daily request limit exceeded. Please try again tomorrow.'}
